# Tensorflow intro

This tutorial shows the basic usage of tensorflow to train neural networks

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Simple NN for classification

## Data processing
Read data and convert them to numerical inputs

In [3]:
X, y = make_classification(n_samples=20000, n_features=8, n_informative=5, 
                           n_redundant=0, n_classes=2, random_state=1)

In [4]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
print('train size: {}, target_ratio: {:.3f}'.format(X_train.shape, np.mean(y_train)))
print('test size: {}, target_ratio: {:.3f}'.format(X_test.shape, np.mean(y_test)))
print('dev size: {}, target_ratio: {:.3f}'.format(X_dev.shape, np.mean(y_dev)))

train size: (12800, 8), target_ratio: 0.502
test size: (4000, 8), target_ratio: 0.501
dev size: (3200, 8), target_ratio: 0.502


## Building a simple model with tf.keras

Very useful documentations with many examples and detailed explanation of everything you might need:
 - https://www.tensorflow.org/api_docs/python/tf/keras/
 - https://keras.io/api/

Contain everything about:
  - Model building: Activations, Losses, Optimizers, Regularization
  - Data processing
  - Pretrained models and datasets
  - Automatic differentiation
  - ...

  

### Model speficication

three APIs for building the model
   - sequential - easy to code, but less flexible - we will use it sometimes
   - functional - flexible and still easy to code - we will use it the most
   - model subclassing - rather complicated and not very much used - we will skip it

#### Sequential API

(https://www.tensorflow.org/guide/keras/sequential_model)

Easy to code but <span style="color:red"> NOT </span> appropriate when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [8]:
# Specification A)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([X_train.shape[1],]), # Create input layer with 'input data' neurons
    tf.keras.layers.Dense(10, activation="relu"), # Create hidden layer with 10 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation="sigmoid"), # Create output layer with one neuron and sigmoid activation
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                90        
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Specification B)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                90        
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


#### Functional API

(https://www.tensorflow.org/guide/keras/functional)

The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.



In [10]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

hidden = tf.keras.layers.Dense(10)(inputs)
hidden = tf.keras.activations.relu(hidden)
hidden = tf.keras.layers.Dense(1)(hidden)
outputs = tf.keras.activations.sigmoid(hidden)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='Model')

In [11]:
model.summary()

Model: "Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_6 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 10)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid (TFOpLambda  (None, 1)                0         
 )                                                               
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_______________________________________________________________

In [12]:
# TODO: To make the following line work you need to install graphviz (if you have not done so in one of the previous classes)
# 1) follow the instructions https://graphviz.gitlab.io/download/?fbclid=IwAR1V-lrRhho5rSfBVYXYISsighqRwOCOgMHLmL_DclkQrPtMXQaKj3mFcqs
# 2) this notebook has been tested with version 8.0.3
# 3) make sure you add it to the PATH variable (you are specifically asked during the installation) at least for local user

tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


### Model compilation and training

In [13]:
# compile the model with selected optimizer, loss and metrics
model.compile(
        optimizer=tf.optimizers.Adam(), # Several other possibilities for optimizers 
        loss=tf.losses.BinaryCrossentropy(), # Select the proper loss for the task
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()], # Select the proper metrics for the task
)

In [14]:
print('\n>>> Bias of the last layers:')
print(model.layers[3].weights[1].numpy())

print('\n>>> Kernel of the last layers:')
print(model.layers[3].weights[0].numpy())

print('\n>>> Bias of the first layers:')
print(model.layers[1].weights[1].numpy())

print('\n>>> Kernel of the first layers:')
print(model.layers[1].weights[0].numpy())


>>> Bias of the last layers:
[0.]

>>> Kernel of the last layers:
[[ 0.5154496 ]
 [-0.44701245]
 [-0.237315  ]
 [ 0.01677322]
 [-0.5168031 ]
 [ 0.43923312]
 [-0.02812868]
 [-0.08891332]
 [ 0.5554405 ]
 [ 0.7065024 ]]

>>> Bias of the first layers:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

>>> Kernel of the first layers:
[[-0.04693872  0.5302279  -0.40398994 -0.30423754 -0.5357484   0.19146186
  -0.5093651   0.39954227  0.20027041  0.1854586 ]
 [ 0.32080042  0.27005988  0.25124866  0.37265563  0.2149874  -0.03264832
   0.5160216   0.09610438  0.17622715 -0.0103789 ]
 [-0.17918682 -0.5249128   0.27878904 -0.03069329 -0.3362981   0.06663507
   0.00683314  0.06641293  0.42232203 -0.46182787]
 [ 0.5005971   0.55244124 -0.43924788  0.18193102 -0.27093935 -0.0879297
  -0.07563651 -0.35239404 -0.52682614  0.57553184]
 [-0.25627786  0.5265504  -0.21141422  0.19617432 -0.31979886 -0.22778341
  -0.11741441  0.57247436  0.35500628  0.22974426]
 [-0.29636872  0.11769742 -0.33588243  0.30256796  0.29125166 

In [15]:
# train the model with default setting
model.fit(X_train, y_train, batch_size=64, epochs=100)

Epoch 1/100


2024-04-09 18:15:51.483717: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


200/200 [==============================] - 0s 383us/step - loss: 0.7562 - auc: 0.5022 - binary_accuracy: 0.5065
Epoch 2/100
200/200 [==============================] - 0s 344us/step - loss: 0.5266 - auc: 0.8279 - binary_accuracy: 0.7354
Epoch 3/100
200/200 [==============================] - 0s 347us/step - loss: 0.4262 - auc: 0.8969 - binary_accuracy: 0.8091
Epoch 4/100
200/200 [==============================] - 0s 348us/step - loss: 0.3760 - auc: 0.9206 - binary_accuracy: 0.8420
Epoch 5/100
200/200 [==============================] - 0s 343us/step - loss: 0.3436 - auc: 0.9343 - binary_accuracy: 0.8648
Epoch 6/100
200/200 [==============================] - 0s 341us/step - loss: 0.3194 - auc: 0.9434 - binary_accuracy: 0.8789
Epoch 7/100
200/200 [==============================] - 0s 344us/step - loss: 0.3011 - auc: 0.9494 - binary_accuracy: 0.8890
Epoch 8/100
200/200 [==============================] - 0s 340us/step - loss: 0.2874 - auc: 0.9530 - binary_accuracy: 0.8937
Epoch 9/100
200/200 

In [16]:
# Evaluate the model and predict for the test data
model.evaluate(X_test, y_test)
test_pred = model.predict(X_test)

125/125 [==============================] - 0s 246us/step


In [17]:
for pred, true in zip(test_pred, y_test[0:10]):
    print('{} - {:.2f}'.format(true, pred[0]))

1 - 0.52
0 - 0.32
0 - 0.41
1 - 1.00
0 - 0.00
0 - 0.02
1 - 0.98
0 - 0.00
1 - 0.97
1 - 1.00


### Add early stopping and regularization

In [18]:
# Input layer
inputs = tf.keras.Input(shape=(X_train.shape[1]))

# Hidden layer with regularization and ReLU
hidden = tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs)
hidden = tf.keras.activations.relu(hidden)

# Output layer with regularization and sigmoid
outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.001))(hidden)
outputs = tf.keras.activations.sigmoid(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

model.compile(
        optimizer=tf.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()],
)

model.summary()

Model: "RegularizedModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_8 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu_1 (TFOpLambda)   (None, 10)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid_1 (TFOpLamb  (None, 1)                0         
 da)                                                             
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
____________________________________________________

In [19]:
epochs = 200

early_call = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_dev, y_dev),
          callbacks=[early_call])

Epoch 1/200
400/400 [==============================] - 1s 676us/step - loss: 0.6819 - auc_1: 0.6865 - binary_accuracy: 0.6496 - val_loss: 0.4881 - val_auc_1: 0.8718 - val_binary_accuracy: 0.8047
Epoch 2/200
400/400 [==============================] - 0s 413us/step - loss: 0.4440 - auc_1: 0.8919 - binary_accuracy: 0.8208 - val_loss: 0.4020 - val_auc_1: 0.9121 - val_binary_accuracy: 0.8366
Epoch 3/200
400/400 [==============================] - 0s 411us/step - loss: 0.3859 - auc_1: 0.9182 - binary_accuracy: 0.8446 - val_loss: 0.3598 - val_auc_1: 0.9296 - val_binary_accuracy: 0.8631
Epoch 4/200
400/400 [==============================] - 0s 413us/step - loss: 0.3523 - auc_1: 0.9330 - binary_accuracy: 0.8681 - val_loss: 0.3359 - val_auc_1: 0.9396 - val_binary_accuracy: 0.8813
Epoch 5/200
400/400 [==============================] - 0s 411us/step - loss: 0.3293 - auc_1: 0.9426 - binary_accuracy: 0.8836 - val_loss: 0.3198 - val_auc_1: 0.9466 - val_binary_accuracy: 0.8900
Epoch 6/200
400/400 [====

### Gridsearch and Tensorboard
Run gridsearch over hidden layer size, L2 regularization, activation, check the outputs in Tensorboard

I recommend not to run Tensorboard from Jupyter notebook but from terminal directly

use "tensorboard --logdir logs" in command line

In [20]:
# hidden_sizes = [2, 5, 10, 20, 50]
# l2_regs = [0.01, 0.001, 0.0001]
# activations = ['relu', 'tanh']

hidden_sizes = [2, 5]
l2_regs = [0.01]
activations = ['relu', 'tanh']

epochs = 10
batch_size = 64

early_call = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', patience=10, restore_best_weights=True)

for activation in activations:
    for l2_reg in l2_regs:
        for hidden_size in hidden_sizes:
            if activation == 'relu':
                activate = tf.keras.activations.relu
            elif activation == 'tanh':
                activate = tf.keras.activations.tanh

            # Create Tensorboard Callback
            param_string = 'act-{},l2-{},hs-{}'.format(activation, l2_reg, hidden_size)
            log_dir = 'logs/binary_classification_test/' + param_string
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            # Input layer
            inputs = tf.keras.Input(shape=(X_train.shape[1]))

            # Hidden layer with regularization and ReLU
            hidden = tf.keras.layers.Dense(hidden_size, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs)
            hidden = activate(hidden)

            # Output layer with regularization and sigmoid
            outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(hidden)
            outputs = tf.keras.activations.sigmoid(outputs)

            model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

            model.compile(
                    optimizer=tf.optimizers.Adam(),
                    loss=tf.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.AUC(name='AUC'), tf.keras.metrics.BinaryAccuracy()],
            )

            # Train the model
            model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, y_dev),
                      callbacks=[early_call, tensorboard_callback])

Epoch 1/10
200/200 [==============================] - 1s 1ms/step - loss: 1.0894 - AUC: 0.3901 - binary_accuracy: 0.4209 - val_loss: 0.9411 - val_AUC: 0.3918 - val_binary_accuracy: 0.4253
Epoch 2/10
200/200 [==============================] - 0s 478us/step - loss: 0.8571 - AUC: 0.4234 - binary_accuracy: 0.4494 - val_loss: 0.7966 - val_AUC: 0.4360 - val_binary_accuracy: 0.4628
Epoch 3/10
200/200 [==============================] - 0s 480us/step - loss: 0.7598 - AUC: 0.4681 - binary_accuracy: 0.4938 - val_loss: 0.7302 - val_AUC: 0.4953 - val_binary_accuracy: 0.5172
Epoch 4/10
200/200 [==============================] - 0s 481us/step - loss: 0.7084 - AUC: 0.5268 - binary_accuracy: 0.5459 - val_loss: 0.6892 - val_AUC: 0.5620 - val_binary_accuracy: 0.5775
Epoch 5/10
200/200 [==============================] - 0s 487us/step - loss: 0.6731 - AUC: 0.5926 - binary_accuracy: 0.6101 - val_loss: 0.6579 - val_AUC: 0.6327 - val_binary_accuracy: 0.6422
Epoch 6/10
200/200 [==============================] 

In [26]:
%reload_ext tensorboard
%tensorboard --logdir ./logs

Reusing TensorBoard on port 6006 (pid 63394), started 0:01:15 ago. (Use '!kill 63394' to kill it.)